Import Packages

In [ ]:
import pandas as pd
import numpy as np
from pyampute.exploration.mcar_statistical_tests import MCARTest
from sklearn.impute import KNNImputer
import scipy as sp

Read Data and format numbers

In [ ]:
columns=['Entity', 'Year', 'Access to electricity (% of population)', 'Access to clean fuels for cooking', 'Renewable-electricity-generating-capacity-per-capita', 'Financial flows to developing countries (US $)', 'Renewable energy share in the total final energy consumption (%)', 'Electricity from fossil fuels (TWh)', 'Electricity from nuclear (TWh)', 'Electricity from renewables (TWh)', 'Low-carbon electricity (% electricity)', 'Primary energy consumption per capita (kWh/person)', 'Energy intensity level of primary energy (MJ/$2017 PPP GDP)', 'Value_co2_emissions_kt_by_country', 'Renewables (% equivalent primary energy)', 'gdp_growth', 'gdp_per_capita', 'Density\\n(P/Km2)', 'Land Area(Km2)', 'Latitude', 'Longitude']

def convert_to_float(value):
    if isinstance(value, str) and ',' in value:
        return float(value.replace(',', '.'))
    return value

data = pd.read_csv('./co2_emissions/emission_data.csv')
pd.DataFrame(data)

Determine the number of NaNs

In [ ]:
nan_counts = data.isna().sum() *100 / len(data)
print(nan_counts)

Can MCAR Methods be used?

In [ ]:
data_no_countries = pd.read_table('./co2_emissions/emission_data_modified.csv', sep=',')
mt = MCARTest(method="little")
if mt.little_mcar_test(data_no_countries) > 0.05:
    print('Is MCAR')
else:
    print('Not MCAR')

Data is not MCAR so KNN can't really be used, but it can be a solution to try with the dataset anyways

In [ ]:
knn_imputer = KNNImputer(n_neighbors=1)
imputed_data = knn_imputer.fit_transform(data_no_countries)
pd.DataFrame(imputed_data, columns=columns[2:])

Pipeline v2
Because intellij lost all my code :((

Set all groups with all NaNs to 0

In [ ]:
groups_with_all_nans = data.groupby("Entity").apply(lambda grp: grp.isna().all())
for grp in groups_with_all_nans:
    print(grp)
    # if groups_with_all_nans[grp[0], grp[1]]:
    #     data.loc[grp[0], grp[1]] = 0

Count the number of zeros

In [ ]:
num_zeros = (data == 0).sum()*100 / len(data)
print(num_zeros)

Remove Columns with more than 20% zeros as they cannot be good reference for the entire set

In [ ]:
data.drop('Electricity from nuclear (TWh)', axis=1)
pd.DataFrame(data)